In [ ]:
!pip install face_recognition

In [ ]:
import json
from google.colab import files

class MissingPersonDatabase:
    def __init__(self):
        self.database = {}

    def add_person(self, name, current_photo, childhood_photo, info):
        self.database[name] = {'current_photo': current_photo,
                               'childhood_photo': childhood_photo,
                               'info': info}
        self.save_to_file()

    def save_to_file(self, filename='missing_person_database.json'):
        with open(filename, 'w') as file:
            json.dump(self.database, file, indent=4)

# Create an instance of the MissingPersonDatabase
database = MissingPersonDatabase()

# Function to handle adding a missing person
def add_missing_person():
    name = input("Enter person's name: ")
    current_photo = files.upload()
    childhood_photo = files.upload()
    age = input("Enter person's age: ")
    contact = input("Enter person's contact information: ")
    info = {'age': age, 'contact': contact}
    database.add_person(name, list(current_photo.keys())[0], list(childhood_photo.keys())[0], info)

# Function to add multiple missing persons
def add_multiple_missing_persons():
    add_more = True
    while add_more:
        add_missing_person()
        choice = input("Do you want to add another missing person? (yes/no): ").lower()
        if choice != 'yes':
            add_more = False

# Add missing persons to the database
add_multiple_missing_persons()


In [ ]:
import cv2
import face_recognition
import numpy as np
import json
from google.colab import files

class MissingPersonDatabase:
    def __init__(self):
        self.database = {}

    def load_from_file(self, filename='missing_person_database.json'):
        try:
            with open(filename, 'r') as file:
                self.database = json.load(file)
        except FileNotFoundError:
            print("Database file not found. Please make sure to create the database first.")

    def search_person(self, query_img):
        if query_img is None:
            return None, None

        # Load known face encodings and names
        known_face_encodings = []
        known_face_names = []
        for name, data in self.database.items():
            current_img = face_recognition.load_image_file(data['current_photo'])
            childhood_img = face_recognition.load_image_file(data['childhood_photo'])
            current_face_encoding = face_recognition.face_encodings(current_img)
            childhood_face_encoding = face_recognition.face_encodings(childhood_img)
            if current_face_encoding:
                known_face_encodings.append(current_face_encoding[0])
                known_face_names.append(name)
            if childhood_face_encoding:
                known_face_encodings.append(childhood_face_encoding[0])
                known_face_names.append(name)

        # Find matches
        face_locations = face_recognition.face_locations(query_img)
        query_face_encodings = face_recognition.face_encodings(query_img, face_locations)
        best_match = None
        min_distance = float('inf')
        for query_face_encoding in query_face_encodings:
            distances = face_recognition.face_distance(known_face_encodings, query_face_encoding)
            for i, distance in enumerate(distances):
                if distance < min_distance:
                    min_distance = distance
                    best_match = (known_face_names[i], self.database[known_face_names[i]], distance)
        if best_match and 1 - best_match[2] > 0.35:  # Check if similarity score is above threshold
            return best_match[0], best_match[1], 1 - best_match[2]  # Name, Info, Similarity Score
        return None, None, 1 - min_distance  # No match found, return None for name and info

# Create an instance of the MissingPersonDatabase
database = MissingPersonDatabase()

# Load the database from file
database.load_from_file()

# Function to handle file upload and search
def handle_upload():
    uploaded = files.upload()
    for filename in uploaded.keys():
        query_img = cv2.imdecode(np.frombuffer(uploaded[filename], np.uint8), cv2.IMREAD_COLOR)
        name, info, similarity = database.search_person(query_img)
        if name:
            print("Match found:")
            print("Name:", name)
            print("Age:", info['info']['age'])
            print("Contact:", info['info']['contact'])
            print("Similarity Score:", similarity)
        else:
            print("No match found.")
            print("Lowest Similarity Score:", similarity)

# Perform file upload and search
print("Upload the photo to search:")
handle_upload()